In [1]:
"""
Veri seti yüklendi, sınıflar ve özelliklerin başlıkları güncellendi.
"""

import pandas as pd
df = pd.read_csv("../data/drug_consumption.data", header=None)
new_columns = [
    "ID", "Age", "Gender", "Education", "Country", "Ethnicity",
    "Nscore", "Escore", "Oscore", "Ascore", "Cscore",
    "Impulsiveness", "SS", "Alcohol", "Amphetamines", "Amyl_nitrite",
    "Benzodiazepines", "Cannabis", "Chocolate", "Cocaine", "Caffeine",
    "Crack", "Ecstasy", "Heroin", "Ketamine", "Legal_highs", "LSD",
    "Methadone", "Mushrooms", "Nicotine", "VSA", "Semeron"
]

df.columns = new_columns

In [2]:
"""
Madde kullanım süreleri, numerik hale getirildi.

0 (Veri Setinde: CL0): Hiç Kullanmamış
1 (Veri Setinde: CL1): 10+ Yıl Önce Kullanmış
2 (Veri Setinde: CL2): Son 10 Yıl İçinde Kullanmış
3 (Veri Setinde: CL3): Son 1 Yıl İçinde Kullanmış
4 (Veri Setinde: CL4): Son 1 Ay İçinde Kullanmış
5 (Veri Setinde: CL5): Son 1 Hafta İçinde Kullanmış
6 (Veri Setinde: CL6): Son 24 Saat İçinde Kullanmış

"""

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in df.columns:
    if df[col].dtype == 'object' and col not in ["ID", "Cocaine_binary"]:
        df[col] = le.fit_transform(df[col])


In [3]:
"""

Kokain, araştırmaya konu olan uyuşturucu olarak seçildi.
Kokain Kullanım Durumunu gösteren bir sınıf oluşturuldu (binary class).
[0,1]: Kullanmıyor - 0,
[2,6]: Kullanıyor - 1.

"""

df["Cocaine_binary"] = df["Cocaine"].apply(
    lambda x: 1 if x >= 2 else 0
)

In [4]:
"""
ID Özelliği çıkarıldı, Özellik seçimi ve modellerin eğitimi için gereken train-test ayrımı yapıldı.
X: Bağımsız Değişken(ler)
Y: Bağımlı Değişken 

"""

from sklearn.model_selection import train_test_split

X = df.drop(["Cocaine", "Cocaine_binary", "ID"], axis=1)
y = df["Cocaine_binary"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

In [29]:
"""

Feature Selection 1: Information Gain ile En Değerli Özellikleri Belirleme

Entropi ve Information Gain hesapları yapıldı.
En yüksek gain'li 20 feature listelendi.


"""

from sklearn.feature_selection import mutual_info_classif

mi_scores = mutual_info_classif(X, y, random_state=42)
mi_df = pd.DataFrame({
    "feature": X.columns,
    "mi_score": mi_scores
}).sort_values(by="mi_score", ascending=False)

print("Information Gain algoritmasının en değerli bulduğu 20 özellik: ")
mi_df.head(20)

Information Gain algoritmasının en değerli bulduğu 20 özellik: 


,feature,mi_score
4,Ethnicity,0.005860
7,Oscore,0.005546
3,Country,0.005150
1,Gender,0.004134
15,Benzodiazepines,0.002768
9,Cscore,0.002151
27,Nicotine,0.002102
24,LSD,0.001977
10,Impulsiveness,0.001962
8,Ascore,0.001922


In [30]:
"""

Feature Selection 2: ReliefF ile En İyi Ayrım Yaptırmaya Yarayan Özellikleri Belirleme

En fazla ağırlığa sahip olan 20 feature listelendi.

"""

from skrebate import ReliefF

relief = ReliefF(n_neighbors=100)
relief.fit(X_train.values, y_train.values)

relief_scores = relief.feature_importances_

relief_df = pd.DataFrame({
    "feature": X.columns,
    "relief_score": relief_scores
}).sort_values(by="relief_score", ascending=False)

print("Information Gain algoritmasının en değerli bulduğu 20 özellik: ")
relief_df.head(20)

Information Gain algoritmasının en değerli bulduğu 20 özellik: 


,feature,relief_score
3,Country,0.315387
0,Age,0.310868
26,Mushrooms,0.303354
17,Chocolate,0.290906
18,Caffeine,0.274010
15,Benzodiazepines,0.268111
24,LSD,0.262536
16,Cannabis,0.249947
1,Gender,0.248488
13,Amphetamines,0.248212


In [7]:
""" 
Min-Max Normalizasyonu ile değerleri 0-1 arasına çekelim: Ki-Kare Testi için gerekli.
"""

from sklearn.preprocessing import MinMaxScaler

scaler_for_chisq = MinMaxScaler()
X_scaled_fs = scaler_for_chisq.fit_transform(X)

In [8]:
"""

Feature Selection 3: Ki-Kare Testi.

"""

from sklearn.feature_selection import SelectKBest, chi2

chi2_selector = SelectKBest(score_func=chi2, k='all')
chi2_selector.fit(X_scaled_fs, y)
chi2_scores = chi2_selector.scores_

In [31]:
"""

Feature Selection 3: Ki-Kare Testi.

Feature'lar ile hedef değişken arasındaki ilişkilerin gösterimi gerçekleştiriliyor.
En güçlü ilişkiye sahip 20 feature listelendi.

"""

chi2_df = pd.DataFrame({
    "feature": X.columns,
    "chi2_score": chi2_scores
}).sort_values(by="chi2_score", ascending=False)

print("Ki-Kare Testi algoritmasının en değerli bulduğu 20 özellik: ")
chi2_df.head(20)

Ki-Kare Testi algoritmasının en değerli bulduğu 20 özellik: 


,feature,chi2_score
28,VSA,5.416264
20,Ecstasy,1.627103
19,Crack,1.497346
3,Country,1.352482
17,Chocolate,1.066078
21,Heroin,0.811662
22,Ketamine,0.678674
24,LSD,0.675168
12,Alcohol,0.517096
16,Cannabis,0.506823


In [10]:
"""
Modellerin çalışması için gerekli olan algoritmalar ve
yorumlamalar için gereken metrikler ithal ediliyor
"""

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [11]:
"""
Lojistik Regresyon ve Destek Vektörü Makinesi Algoritmaları 
için gerekli olan ölçeklenmiş veriler hazırlanıyor
"""
scaler_model = StandardScaler()
X_train_scaled = scaler_model.fit_transform(X_train)
X_test_scaled = scaler_model.transform(X_test)

In [12]:
"""
Sonuçları ekrana yazdıracak fonksiyon
"""

def bilgi_yazdir(name, y_true, y_pred):
    print(f"\n{name} Sonuçları\n")
    print("--------------------------")
    print(f"Accuracy Skoru: %{accuracy_score(y_true, y_pred) * 100:.2f} ")
    print(f"Precision Skoru: %{precision_score(y_true, y_pred) * 100:.2f}")
    print(f"Recall Skoru: %{recall_score(y_true, y_pred) * 100:.2f}")
    print(f"F1 Skoru: %{f1_score(y_true, y_pred) * 100:.2f}")
    print("Confusion Matrisi:\n", confusion_matrix(y_true, y_pred))
    print("--------------------------")

In [13]:
"""
Lojistik Regresyon Algoritması 1: Feature Selection İşlemleri Gerçekleştirilmemiş Versiyon
"""

lojistik_regresyon = LogisticRegression(
    max_iter=500,
    class_weight="balanced", 
    random_state=42
)
lojistik_regresyon.fit(X_train_scaled, y_train)
y_tahmin_lr = lojistik_regresyon.predict(X_test_scaled)

In [14]:
"""
Destek Vektörü Makinesi Algoritması 1: Feature Selection İşlemleri Gerçekleştirilmemiş Versiyon
"""
destek_vektoru = SVC(
    kernel="rbf",
    class_weight="balanced",
    probability=False,
    random_state=42
)

destek_vektoru.fit(X_train_scaled, y_train)
y_tahmin_svm = destek_vektoru.predict(X_test_scaled)

In [15]:
"""
Rassal Orman Algoritması 1: Feature Selection İşlemleri Gerçekleştirilmemiş Versiyon
"""

rassal_orman = RandomForestClassifier(
    n_estimators=300,
    random_state=42,
    class_weight="balanced"
)

rassal_orman.fit(X_train, y_train)
y_tahmin_rf = rassal_orman.predict(X_test)

In [16]:
"""
Eğitim veri seti, FS algoritmalarının belirlediği 9 adet ortak
en düşük değere sahip feature'lar silinecek şekilde düzenleniyor
"""

chi2_en_dusukler = chi2_df.sort_values(by="chi2_score").head(20)["feature"].tolist()

relief_en_dusukler = relief_df.sort_values(by="relief_score").head(20)["feature"].tolist()

mi_en_dusukler = mi_df.sort_values(by="mi_score").head(20)["feature"].tolist()

ortak_dusukler = set(chi2_en_dusukler) & set(relief_en_dusukler) & set(mi_en_dusukler)

print("FS Algoritmalarının ortak olarak gösterdiği en az değerli feature'lar: ")
for c in ortak_dusukler:
    print(f"{c}, ")

FS Algoritmalarının ortak olarak gösterdiği en az değerli feature'lar: 
SS, 
Education, 
Legal_highs, 
Nscore, 
Semeron, 
Escore, 
Amyl_nitrite, 
Methadone, 


In [17]:
"""
En az değerli feature'lar silinerek yeni bir eğitim seti oluşturuluyor
"""

silinecek = list(ortak_dusukler)
final_drop_list = [col for col in silinecek + ["Cocaine", "Cocaine_binary"] if col in df.columns]
X_reduced = df.drop(columns=final_drop_list)

In [18]:
"""
Yeni train_test_split gerçekleştiriliyor
"""

X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(
    X_reduced, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

In [19]:
"""
Azaltılmış veri setinin ölçeklenme işlemleri gerçekleştiriliyor
"""
scaler_r = StandardScaler()
X_train_r_scaled = scaler_r.fit_transform(X_train_r)
X_test_r_scaled = scaler_r.transform(X_test_r)

In [20]:
"""
Lojistik Regresyon Algoritması 2: Feature Selection Sonrası Versiyon
"""
lojistik_regresyon_reduced = LogisticRegression(
    max_iter=500,
    class_weight="balanced",
    random_state=42
)

lojistik_regresyon_reduced.fit(X_train_r_scaled, y_train_r)
y_tahmin_lr_reduced = lojistik_regresyon_reduced.predict(X_test_r_scaled)

In [21]:
"""
Destek Vektör Makinesi Algoritması 2: Feature Selection Sonrası Versiyon
"""

destek_vektoru_reduced = SVC(
    kernel="rbf",
    class_weight="balanced",
    random_state=42
)

destek_vektoru_reduced.fit(X_train_r_scaled, y_train_r)
y_tahmin_svm_reduced = destek_vektoru_reduced.predict(X_test_r_scaled)

In [22]:
"""
Rassal Orman Algoritması 2: Feature Selection Sonrası Versiyon
"""

rassal_orman_reduced = RandomForestClassifier(
    n_estimators=300,
    class_weight="balanced",
    random_state=42
)

rassal_orman_reduced.fit(X_train_r, y_train_r)
y_tahmin_rf_reduced = rassal_orman_reduced.predict(X_test_r)

In [23]:
""" 
Algoritmalar ve yaptıkları tahminlemeler eşleştiriliyor
"""

tahminler_fs_oncesi = {
    "Lojistik Regresyon (FS Öncesi)": y_tahmin_lr, 
    "Destek Vektör Makinesi (FS Öncesi)": y_tahmin_svm, 
    "Rassal Orman (FS Öncesi)": y_tahmin_rf 
}
tahminler_fs_sonrasi = {
    "Lojistik Regresyon (FS Sonrası)": y_tahmin_lr_reduced,
    "Destek Vektör Makinesi (FS Sonrası)": y_tahmin_svm_reduced,
    "Rassal Orman (FS Sonrası)": y_tahmin_rf_reduced
}

In [24]:
"""
Feature Selection işlemlerinden önceki tahminlemeler ve skorlar:
"""
for model_adi, tahmin in tahminler_fs_oncesi.items():
    bilgi_yazdir(model_adi, y_test, tahmin)


Lojistik Regresyon (FS Öncesi) Sonuçları

--------------------------
Accuracy Skoru: %79.05 
Precision Skoru: %98.34
Recall Skoru: %80.00
F1 Skoru: %88.23
Confusion Matrisi:
 [[  2   5]
 [ 74 296]]
--------------------------

Destek Vektör Makinesi (FS Öncesi) Sonuçları

--------------------------
Accuracy Skoru: %94.43 
Precision Skoru: %98.07
Recall Skoru: %96.22
F1 Skoru: %97.14
Confusion Matrisi:
 [[  0   7]
 [ 14 356]]
--------------------------

Rassal Orman (FS Öncesi) Sonuçları

--------------------------
Accuracy Skoru: %98.14 
Precision Skoru: %98.14
Recall Skoru: %100.00
F1 Skoru: %99.06
Confusion Matrisi:
 [[  0   7]
 [  0 370]]
--------------------------


In [25]:
"""
Feature Selection işlemlerinden sonraki tahminlemeler ve skorlar:
"""

for model_adi, tahmin in tahminler_fs_sonrasi.items():
    bilgi_yazdir(model_adi, y_test_r, tahmin)


Lojistik Regresyon (FS Sonrası) Sonuçları

--------------------------
Accuracy Skoru: %72.94 
Precision Skoru: %98.20
Recall Skoru: %73.78
F1 Skoru: %84.26
Confusion Matrisi:
 [[  2   5]
 [ 97 273]]
--------------------------

Destek Vektör Makinesi (FS Sonrası) Sonuçları

--------------------------
Accuracy Skoru: %94.43 
Precision Skoru: %98.07
Recall Skoru: %96.22
F1 Skoru: %97.14
Confusion Matrisi:
 [[  0   7]
 [ 14 356]]
--------------------------

Rassal Orman (FS Sonrası) Sonuçları

--------------------------
Accuracy Skoru: %98.14 
Precision Skoru: %98.14
Recall Skoru: %100.00
F1 Skoru: %99.06
Confusion Matrisi:
 [[  0   7]
 [  0 370]]
--------------------------
